# Notebook for Prompt Analysis for CLIP using Mapper

In [1]:
import os
import numpy as np
from pathlib import Path

import torch
import clip
import kmapper as km
from sklearn import cluster
import gradio as gr
from PIL import Image
from kmapper.plotlyviz import plotlyviz
from matplotlib import pyplot as plt

/home/ameya/Projects/venv/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Loading the CLIP model

In [2]:
model_types = clip.available_models()

In [3]:
model, preprocess = clip.load("ViT-B/32")

## Creating Imagenette templates and prompts

In [4]:
#imagenet_classes = ["tench", "goldfish", "great white shark", "tiger shark", "hammerhead shark", "electric ray", "stingray", "rooster", "hen", "ostrich", "brambling", "goldfinch", "house finch", "junco", "indigo bunting", "American robin", "bulbul", "jay", "magpie", "chickadee", "American dipper", "kite (bird of prey)", "bald eagle", "vulture", "great grey owl", "fire salamander", "smooth newt", "newt", "spotted salamander", "axolotl", "American bullfrog", "tree frog", "tailed frog", "loggerhead sea turtle", "leatherback sea turtle", "mud turtle", "terrapin", "box turtle", "banded gecko", "green iguana", "Carolina anole", "desert grassland whiptail lizard", "agama", "frilled-necked lizard", "alligator lizard", "Gila monster", "European green lizard", "chameleon", "Komodo dragon", "Nile crocodile", "American alligator", "triceratops", "worm snake", "ring-necked snake", "eastern hog-nosed snake", "smooth green snake", "kingsnake", "garter snake", "water snake", "vine snake", "night snake", "boa constrictor", "African rock python", "Indian cobra", "green mamba", "sea snake", "Saharan horned viper", "eastern diamondback rattlesnake", "sidewinder rattlesnake", "trilobite", "harvestman", "scorpion", "yellow garden spider", "barn spider", "European garden spider", "southern black widow", "tarantula", "wolf spider", "tick", "centipede", "black grouse", "ptarmigan", "ruffed grouse", "prairie grouse", "peafowl", "quail", "partridge", "african grey parrot", "macaw", "sulphur-crested cockatoo", "lorikeet", "coucal", "bee eater", "hornbill", "hummingbird", "jacamar", "toucan", "duck", "red-breasted merganser", "goose", "black swan", "tusker", "echidna", "platypus", "wallaby", "koala", "wombat", "jellyfish", "sea anemone", "brain coral", "flatworm", "nematode", "conch", "snail", "slug", "sea slug", "chiton", "chambered nautilus", "Dungeness crab", "rock crab", "fiddler crab", "red king crab", "American lobster", "spiny lobster", "crayfish", "hermit crab", "isopod", "white stork", "black stork", "spoonbill", "flamingo", "little blue heron", "great egret", "bittern bird", "crane bird", "limpkin", "common gallinule", "American coot", "bustard", "ruddy turnstone", "dunlin", "common redshank", "dowitcher", "oystercatcher", "pelican", "king penguin", "albatross", "grey whale", "killer whale", "dugong", "sea lion", "Chihuahua", "Japanese Chin", "Maltese", "Pekingese", "Shih Tzu", "King Charles Spaniel", "Papillon", "toy terrier", "Rhodesian Ridgeback", "Afghan Hound", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "English foxhound", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Italian Greyhound", "Whippet", "Ibizan Hound", "Norwegian Elkhound", "Otterhound", "Saluki", "Scottish Deerhound", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Irish Terrier", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Australian Terrier", "Dandie Dinmont Terrier", "Boston Terrier", "Miniature Schnauzer", "Giant Schnauzer", "Standard Schnauzer", "Scottish Terrier", "Tibetan Terrier", "Australian Silky Terrier", "Soft-coated Wheaten Terrier", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "German Shorthaired Pointer", "Vizsla", "English Setter", "Irish Setter", "Gordon Setter", "Brittany dog", "Clumber Spaniel", "English Springer Spaniel", "Welsh Springer Spaniel", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Groenendael dog", "Malinois", "Briard", "Australian Kelpie", "Komondor", "Old English Sheepdog", "Shetland Sheepdog", "collie", "Border Collie", "Bouvier des Flandres dog", "Rottweiler", "German Shepherd Dog", "Dobermann", "Miniature Pinscher", "Greater Swiss Mountain Dog", "Bernese Mountain Dog", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Tibetan Mastiff", "French Bulldog", "Great Dane", "St. Bernard", "husky", "Alaskan Malamute", "Siberian Husky", "Dalmatian", "Affenpinscher", "Basenji", "pug", "Leonberger", "Newfoundland dog", "Great Pyrenees dog", "Samoyed", "Pomeranian", "Chow Chow", "Keeshond", "brussels griffon", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Toy Poodle", "Miniature Poodle", "Standard Poodle", "Mexican hairless dog (xoloitzcuintli)", "grey wolf", "Alaskan tundra wolf", "red wolf or maned wolf", "coyote", "dingo", "dhole", "African wild dog", "hyena", "red fox", "kit fox", "Arctic fox", "grey fox", "tabby cat", "tiger cat", "Persian cat", "Siamese cat", "Egyptian Mau", "cougar", "lynx", "leopard", "snow leopard", "jaguar", "lion", "tiger", "cheetah", "brown bear", "American black bear", "polar bear", "sloth bear", "mongoose", "meerkat", "tiger beetle", "ladybug", "ground beetle", "longhorn beetle", "leaf beetle", "dung beetle", "rhinoceros beetle", "weevil", "fly", "bee", "ant", "grasshopper", "cricket insect", "stick insect", "cockroach", "praying mantis", "cicada", "leafhopper", "lacewing", "dragonfly", "damselfly", "red admiral butterfly", "ringlet butterfly", "monarch butterfly", "small white butterfly", "sulphur butterfly", "gossamer-winged butterfly", "starfish", "sea urchin", "sea cucumber", "cottontail rabbit", "hare", "Angora rabbit", "hamster", "porcupine", "fox squirrel", "marmot", "beaver", "guinea pig", "common sorrel horse", "zebra", "pig", "wild boar", "warthog", "hippopotamus", "ox", "water buffalo", "bison", "ram (adult male sheep)", "bighorn sheep", "Alpine ibex", "hartebeest", "impala (antelope)", "gazelle", "arabian camel", "llama", "weasel", "mink", "European polecat", "black-footed ferret", "otter", "skunk", "badger", "armadillo", "three-toed sloth", "orangutan", "gorilla", "chimpanzee", "gibbon", "siamang", "guenon", "patas monkey", "baboon", "macaque", "langur", "black-and-white colobus", "proboscis monkey", "marmoset", "white-headed capuchin", "howler monkey", "titi monkey", "Geoffroy's spider monkey", "common squirrel monkey", "ring-tailed lemur", "indri", "Asian elephant", "African bush elephant", "red panda", "giant panda", "snoek fish", "eel", "silver salmon", "rock beauty fish", "clownfish", "sturgeon", "gar fish", "lionfish", "pufferfish", "abacus", "abaya", "academic gown", "accordion", "acoustic guitar", "aircraft carrier", "airliner", "airship", "altar", "ambulance", "amphibious vehicle", "analog clock", "apiary", "apron", "trash can", "assault rifle", "backpack", "bakery", "balance beam", "balloon", "ballpoint pen", "Band-Aid", "banjo", "baluster / handrail", "barbell", "barber chair", "barbershop", "barn", "barometer", "barrel", "wheelbarrow", "baseball", "basketball", "bassinet", "bassoon", "swimming cap", "bath towel", "bathtub", "station wagon", "lighthouse", "beaker", "military hat (bearskin or shako)", "beer bottle", "beer glass", "bell tower", "baby bib", "tandem bicycle", "bikini", "ring binder", "binoculars", "birdhouse", "boathouse", "bobsleigh", "bolo tie", "poke bonnet", "bookcase", "bookstore", "bottle cap", "hunting bow", "bow tie", "brass memorial plaque", "bra", "breakwater", "breastplate", "broom", "bucket", "buckle", "bulletproof vest", "high-speed train", "butcher shop", "taxicab", "cauldron", "candle", "cannon", "canoe", "can opener", "cardigan", "car mirror", "carousel", "tool kit", "cardboard box / carton", "car wheel", "automated teller machine", "cassette", "cassette player", "castle", "catamaran", "CD player", "cello", "mobile phone", "chain", "chain-link fence", "chain mail", "chainsaw", "storage chest", "chiffonier", "bell or wind chime", "china cabinet", "Christmas stocking", "church", "movie theater", "cleaver", "cliff dwelling", "cloak", "clogs", "cocktail shaker", "coffee mug", "coffeemaker", "spiral or coil", "combination lock", "computer keyboard", "candy store", "container ship", "convertible", "corkscrew", "cornet", "cowboy boot", "cowboy hat", "cradle", "construction crane", "crash helmet", "crate", "infant bed", "Crock Pot", "croquet ball", "crutch", "cuirass", "dam", "desk", "desktop computer", "rotary dial telephone", "diaper", "digital clock", "digital watch", "dining table", "dishcloth", "dishwasher", "disc brake", "dock", "dog sled", "dome", "doormat", "drilling rig", "drum", "drumstick", "dumbbell", "Dutch oven", "electric fan", "electric guitar", "electric locomotive", "entertainment center", "envelope", "espresso machine", "face powder", "feather boa", "filing cabinet", "fireboat", "fire truck", "fire screen", "flagpole", "flute", "folding chair", "football helmet", "forklift", "fountain", "fountain pen", "four-poster bed", "freight car", "French horn", "frying pan", "fur coat", "garbage truck", "gas mask or respirator", "gas pump", "goblet", "go-kart", "golf ball", "golf cart", "gondola", "gong", "gown", "grand piano", "greenhouse", "radiator grille", "grocery store", "guillotine", "hair clip", "hair spray", "half-track", "hammer", "hamper", "hair dryer", "hand-held computer", "handkerchief", "hard disk drive", "harmonica", "harp", "combine harvester", "hatchet", "holster", "home theater", "honeycomb", "hook", "hoop skirt", "gymnastic horizontal bar", "horse-drawn vehicle", "hourglass", "iPod", "clothes iron", "carved pumpkin", "jeans", "jeep", "T-shirt", "jigsaw puzzle", "rickshaw", "joystick", "kimono", "knee pad", "knot", "lab coat", "ladle", "lampshade", "laptop computer", "lawn mower", "lens cap", "letter opener", "library", "lifeboat", "lighter", "limousine", "ocean liner", "lipstick", "slip-on shoe", "lotion", "music speaker", "loupe magnifying glass", "sawmill", "magnetic compass", "messenger bag", "mailbox", "tights", "one-piece bathing suit", "manhole cover", "maraca", "marimba", "mask", "matchstick", "maypole", "maze", "measuring cup", "medicine cabinet", "megalith", "microphone", "microwave oven", "military uniform", "milk can", "minibus", "miniskirt", "minivan", "missile", "mitten", "mixing bowl", "mobile home", "ford model t", "modem", "monastery", "monitor", "moped", "mortar and pestle", "graduation cap", "mosque", "mosquito net", "vespa", "mountain bike", "tent", "computer mouse", "mousetrap", "moving van", "muzzle", "metal nail", "neck brace", "necklace", "baby pacifier", "notebook computer", "obelisk", "oboe", "ocarina", "odometer", "oil filter", "pipe organ", "oscilloscope", "overskirt", "bullock cart", "oxygen mask", "product packet / packaging", "paddle", "paddle wheel", "padlock", "paintbrush", "pajamas", "palace", "pan flute", "paper towel", "parachute", "parallel bars", "park bench", "parking meter", "railroad car", "patio", "payphone", "pedestal", "pencil case", "pencil sharpener", "perfume", "Petri dish", "photocopier", "plectrum", "Pickelhaube", "picket fence", "pickup truck", "pier", "piggy bank", "pill bottle", "pillow", "ping-pong ball", "pinwheel", "pirate ship", "drink pitcher", "block plane", "planetarium", "plastic bag", "plate rack", "farm plow", "plunger", "Polaroid camera", "pole", "police van", "poncho", "pool table", "soda bottle", "plant pot", "potter's wheel", "power drill", "prayer rug", "printer", "prison", "missile", "projector", "hockey puck", "punching bag", "purse", "quill", "quilt", "race car", "racket", "radiator", "radio", "radio telescope", "rain barrel", "recreational vehicle", "fishing casting reel", "reflex camera", "refrigerator", "remote control", "restaurant", "revolver", "rifle", "rocking chair", "rotisserie", "eraser", "rugby ball", "ruler measuring stick", "sneaker", "safe", "safety pin", "salt shaker", "sandal", "sarong", "saxophone", "scabbard", "weighing scale", "school bus", "schooner", "scoreboard", "CRT monitor", "screw", "screwdriver", "seat belt", "sewing machine", "shield", "shoe store", "shoji screen / room divider", "shopping basket", "shopping cart", "shovel", "shower cap", "shower curtain", "ski", "balaclava ski mask", "sleeping bag", "slide rule", "sliding door", "slot machine", "snorkel", "snowmobile", "snowplow", "soap dispenser", "soccer ball", "sock", "solar thermal collector", "sombrero", "soup bowl", "keyboard space bar", "space heater", "space shuttle", "spatula", "motorboat", "spider web", "spindle", "sports car", "spotlight", "stage", "steam locomotive", "through arch bridge", "steel drum", "stethoscope", "scarf", "stone wall", "stopwatch", "stove", "strainer", "tram", "stretcher", "couch", "stupa", "submarine", "suit", "sundial", "sunglasses", "sunglasses", "sunscreen", "suspension bridge", "mop", "sweatshirt", "swim trunks / shorts", "swing", "electrical switch", "syringe", "table lamp", "tank", "tape player", "teapot", "teddy bear", "television", "tennis ball", "thatched roof", "front curtain", "thimble", "threshing machine", "throne", "tile roof", "toaster", "tobacco shop", "toilet seat", "torch", "totem pole", "tow truck", "toy store", "tractor", "semi-trailer truck", "tray", "trench coat", "tricycle", "trimaran", "tripod", "triumphal arch", "trolleybus", "trombone", "hot tub", "turnstile", "typewriter keyboard", "umbrella", "unicycle", "upright piano", "vacuum cleaner", "vase", "vaulted or arched ceiling", "velvet fabric", "vending machine", "vestment", "viaduct", "violin", "volleyball", "waffle iron", "wall clock", "wallet", "wardrobe", "military aircraft", "sink", "washing machine", "water bottle", "water jug", "water tower", "whiskey jug", "whistle", "hair wig", "window screen", "window shade", "Windsor tie", "wine bottle", "airplane wing", "wok", "wooden spoon", "wool", "split-rail fence", "shipwreck", "sailboat", "yurt", "website", "comic book", "crossword", "traffic or street sign", "traffic light", "dust jacket", "menu", "plate", "guacamole", "consomme", "hot pot", "trifle", "ice cream", "popsicle", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "mashed potatoes", "cabbage", "broccoli", "cauliflower", "zucchini", "spaghetti squash", "acorn squash", "butternut squash", "cucumber", "artichoke", "bell pepper", "cardoon", "mushroom", "Granny Smith apple", "strawberry", "orange", "lemon", "fig", "pineapple", "banana", "jackfruit", "cherimoya (custard apple)", "pomegranate", "hay", "carbonara", "chocolate syrup", "dough", "meatloaf", "pizza", "pot pie", "burrito", "red wine", "espresso", "tea cup", "eggnog", "mountain", "bubble", "cliff", "coral reef", "geyser", "lakeshore", "promontory", "sandbar", "beach", "valley", "volcano", "baseball player", "bridegroom", "scuba diver", "rapeseed", "daisy", "yellow lady's slipper", "corn", "acorn", "rose hip", "horse chestnut seed", "coral fungus", "agaric", "gyromitra", "stinkhorn mushroom", "earth star fungus", "hen of the woods mushroom", "bolete", "corn cob", "toilet paper"]
imagenette_classes=['cassette player', 'chain saw',
 'church',
 'English springer',
 'french horn',
 'garbage truck',
 'gas pump',
 'golf ball',
 'parachute',
 'tench']

In [5]:
imagenet_templates = [
    'a bad photo of a {}.',
    'a photo of many {}.',
    'a sculpture of a {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of the {}.',
    'a rendering of a {}.',
    'graffiti of a {}.',
    'a bad photo of the {}.',
    'a cropped photo of the {}.',
    'a tattoo of a {}.',
    'the embroidered {}.',
    'a photo of a hard to see {}.',
    'a bright photo of a {}.',
    'a photo of a clean {}.',
    'a photo of a dirty {}.',
    'a dark photo of the {}.',
    'a drawing of a {}.',
    'a photo of my {}.',
    'the plastic {}.',
    'a photo of the cool {}.',
    'a close-up photo of a {}.',
    'a black and white photo of the {}.',
    'a painting of the {}.',
    'a painting of a {}.',
    'a pixelated photo of the {}.',
    'a sculpture of the {}.',
    'a bright photo of the {}.',
    'a cropped photo of a {}.',
    'a plastic {}.',
    'a photo of the dirty {}.',
    'a jpeg corrupted photo of a {}.',
    'a blurry photo of the {}.',
    'a photo of the {}.',
    'a good photo of the {}.',
    'a rendering of the {}.',
    'a {} in a video game.',
    'a photo of one {}.',
    'a doodle of a {}.',
    'a close-up photo of the {}.',
    'a photo of a {}.',
    'the origami {}.',
    'the {} in a video game.',
    'a sketch of a {}.',
    'a doodle of the {}.',
    'a origami {}.',
    'a low resolution photo of a {}.',
    'the toy {}.',
    'a rendition of the {}.',
    'a photo of the clean {}.',
    'a photo of a large {}.',
    'a rendition of a {}.',
    'a photo of a nice {}.',
    'a photo of a weird {}.',
    'a blurry photo of a {}.',
    'a cartoon {}.',
    'art of a {}.',
    'a sketch of the {}.',
    'a embroidered {}.',
    'a pixelated photo of a {}.',
    'itap of the {}.',
    'a jpeg corrupted photo of the {}.',
    'a good photo of a {}.',
    'a plushie {}.',
    'a photo of the nice {}.',
    'a photo of the small {}.',
    'a photo of the weird {}.',
    'the cartoon {}.',
    'art of the {}.',
    'a drawing of the {}.',
    'a photo of the large {}.',
    'a black and white photo of a {}.',
    'the plushie {}.',
    'a dark photo of a {}.',
    'itap of a {}.',
    'graffiti of the {}.',
    'a toy {}.',
    'itap of my {}.',
    'a photo of a cool {}.',
    'a photo of a small {}.',
    'a tattoo of the {}.',
]

## Making class embeddings

In [6]:
 with torch.no_grad():
    zeroshot_weights = []
    for classname in imagenette_classes:
        texts = [template.format(classname) for template in imagenet_templates] #format with class
        texts = clip.tokenize(texts).cuda() #tokenize
        class_embeddings = model.encode_text(texts) #embed with text encoder
        class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
        #class_embedding = class_embeddings.mean(dim=0)
        #class_embedding /= class_embedding.norm()
        class_embedding = class_embeddings
        zeroshot_weights.append(class_embedding)
    zeroshot_weights = torch.stack(zeroshot_weights, dim=2).cuda()

In [7]:
psize, emb_size, num_classes = zeroshot_weights.shape
zeroshot_weights_emb = zeroshot_weights.permute(0,2,1).reshape(psize * num_classes, -1)

In [8]:
zeroshot_weights_emb.shape

torch.Size([800, 512])

## Loading Imagenetv2 dataset

In [9]:
#!pip install -U git+https://github.com/modestyachts/ImageNetV2_pytorch
#from imagenetv2_pytorch import ImageNetV2Dataset
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
image_ds = ImageFolder('imagenette2-160/train/', transform=preprocess)


## Collect exemplar images for each class

In [10]:
imgs = []
for c in imagenette_classes:
    c = c.lower().replace(' ','_')
    print(c)
    class_idx = image_ds.classes.index(c)
    img_idc = np.where(np.array(image_ds.targets) == class_idx)[0].tolist()
    sampler = torch.utils.data.sampler.SubsetRandomSampler(img_idc)
    loader = torch.utils.data.DataLoader(image_ds, batch_size=20, num_workers=2, sampler=sampler)
    for img, t in loader:
        imgs.append(img)
        break
imgs_tens = torch.stack(imgs, dim=0).cuda()

cassette_player
chain_saw
church
english_springer
french_horn
garbage_truck
gas_pump
golf_ball
parachute
tench


In [11]:
imgs_tens.shape

torch.Size([10, 20, 3, 224, 224])

## Image encodings

In [12]:
img_encs_all = []
for c in range(imgs_tens.shape[0]):
    img_encs = model.encode_image(imgs_tens[c])
    img_encs /= img_encs.norm(dim=-1, keepdim=True)
    img_encs_all.append(img_encs)
img_encs_all = torch.stack(img_encs_all, dim=0)

## Predictions and Margins for each class and prompt

In [13]:
logits_all = []
graphs = []
avg_margins = []
for c in range(img_encs_all.shape[0]):
    logits = img_encs_all[c] @ zeroshot_weights_emb.T
    bs, _ = logits.shape
    logits = logits.reshape(bs, psize, num_classes)
    pred_prompts, pred_ind = torch.topk(logits, 2, dim=2)
    correct_prompt_idxs = torch.where(pred_ind[:,:,0] == c)
    margin = pred_prompts[:,: ,0] - pred_prompts[:,:,1]
    avg_margin = margin.mean(dim=0)
    mapper = km.KeplerMapper(verbose=False)
    G = mapper.map(
            avg_margin.cpu().detach().numpy(),
            zeroshot_weights[:,:,c].cpu().detach().numpy(),
            cover= km.Cover(n_cubes=10,
                            perc_overlap=0.1),
            clusterer = cluster.AgglomerativeClustering(n_clusters=2, affinity='cosine', linkage='complete')
        )
    graphs.append(G)
    avg_margins.append(avg_margin.detach().cpu().numpy())
    custom_tooltip_list= []
    result_html = mapper.visualize(G, 
                                   color_values=avg_margin.cpu().detach().numpy(),
                                   color_function_name='Margin',
                                   custom_tooltips = np.array(imagenet_templates),
                                   path_html=f'{imagenette_classes[c]}_margin.html')

## Pre-selection Accuracy

In [14]:
avg_zeroshot = zeroshot_weights[:5].mean(dim=0)
avg_zeroshot/=avg_zeroshot.norm(dim=-1, keepdim=True)

In [15]:
def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]

In [16]:
image_val_ds =  ImageFolder('imagenette2-160/val/', transform=preprocess)

In [17]:
loader = torch.utils.data.DataLoader(image_val_ds, batch_size=32,num_workers=2)

In [18]:
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(loader):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ avg_zeroshot

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

Top-1 accuracy: 98.39
Top-5 accuracy: 99.90


## Post analysis

For each class, we choose the top prompts in the following fashion:
1. Pick simplical complexes with lowest average confidence
2. Drop the prompts with low confidence

In [16]:
def get_avg_embedding_per_class(templates, label):
    print(templates, label)
    label_text = [i.format(label) for i in templates]
    label_text = clip.tokenize(label_text).cuda() #tokenize
    class_embeddings = model.encode_text(label_text) #embed with text encoder
    class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
    class_embedding = class_embeddings.mean(dim=0, keepdim=True)
    class_embedding /= class_embedding.norm(dim=-1, keepdim=True)
    return class_embedding

In [20]:
imagenette_classes=['cassette player', 'chain saw',
 'church',
 'English springer',
 'french horn',
 'garbage truck',
 'gas pump',
 'golf ball',
 'parachute',
 'tench']

In [21]:
templates_cassette = [
    'a black and white photo of a {}',
    'itap of my {}',
    'a bright photo of a {}',
    'a cropped photo of a {}',
    'a low resolution photo of the {}',
    'a photo of a large {}',
    'a photo of a nice {}',
    'a photo of a weird {}',
    'a pixelated photo of a {}',
    'a photo of the nice {}',
    'a dark photo of a {}'
]

In [22]:
templates_chain = [
    'a bad photo of a {}',
    'a low resolution photo of the {}',
    'a cropped photo of a {}',
    'a low resolution photo of a {}',
    'the embroidered {}',
    'a origami {}',
    'a cartoon {}'
]

In [23]:
templates_church = [
    'a rendering of a {}',
    'a bad photo of the {}',
    'a photo of a clean {}',
    'a photo of my {}',
    'a cropped photo of a {}',
    'a photo of one {}',
    'the origami {}',
    'a sketch of a {}',
    'a low resolution photo of a {}'
]

In [24]:
templates_english =[
    'a jpeg corrupted photo of a {}',
    'a dark photo of the {}',
    'a low resolution photo of a {}',
    'a black and white photo of a {}'
]

In [25]:
templates_french = [
    'a black and white photo of a {}',
    'a bad photo of a {}',
    'a low resolution photo of the {}',
    'a bright photo of a {}',
    'a dark photo of the {}',
    'a cropped photo of a {}'
]

In [26]:
templates_garbage = [
    'a drawing of a {}',
    'a black and white photo of a {}',
    'itap of a {}'
]

In [27]:
templates_gas = [
    'a painting of the {}',
    'a photo of the large {}',
]

In [28]:
templates_golf = [
    'a pixelated photo of a {}',
    'a low resolution photo of the {}',
    'a dark photo of the {}',
    'a pixelated photo of the {}'
]

In [29]:
templates_tench =[
    'a photo of a clean {}',
    'a photo of a dirty {}',
    'a photo of my {}',
    'a close-up photo of a {}',
    'itap of the {}',
    'a black and white photo of a {}'
]

In [30]:
templates_parachute = [
    'a black and white photo of a {}',
    'itap of a {}',
    'a bad photo of a {}',
    'a photo of many {}',
    'a photo of a dirty {}',
    'a drawing of a {}',
    'a photo of my {}'
]

In [31]:
temp_list = [templates_cassette, templates_chain, templates_church, templates_english, templates_french, templates_garbage, templates_gas, templates_golf, templates_parachute, templates_tench]

In [32]:
zero_shot_embs_new = [get_avg_embedding_per_class(t_list, l) for t_list, l in zip(temp_list, imagenette_classes)]

['a black and white photo of a {}', 'itap of my {}', 'a bright photo of a {}', 'a cropped photo of a {}', 'a low resolution photo of the {}', 'a photo of a large {}', 'a photo of a nice {}', 'a photo of a weird {}', 'a pixelated photo of a {}', 'a photo of the nice {}', 'a dark photo of a {}'] cassette player
['a bad photo of a {}', 'a low resolution photo of the {}', 'a cropped photo of a {}', 'a low resolution photo of a {}', 'the embroidered {}', 'a origami {}', 'a cartoon {}'] chain saw
['a rendering of a {}', 'a bad photo of the {}', 'a photo of a clean {}', 'a photo of my {}', 'a cropped photo of a {}', 'a photo of one {}', 'the origami {}', 'a sketch of a {}', 'a low resolution photo of a {}'] church
['a jpeg corrupted photo of a {}', 'a dark photo of the {}', 'a low resolution photo of a {}', 'a black and white photo of a {}'] English springer
['a black and white photo of a {}', 'a bad photo of a {}', 'a low resolution photo of the {}', 'a bright photo of a {}', 'a dark photo o

In [33]:
zero_shot_embs_new = torch.stack(zero_shot_embs_new, dim=0).squeeze(1).T

In [34]:
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(loader):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zero_shot_embs_new

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

Top-1 accuracy: 97.35
Top-5 accuracy: 99.87


In [17]:
def filter_prompts(graph, margin_array, label, k=5):
    nodes = graph['nodes']
    scplxes = graph['simplices']
    scplx_margins = []
    for cplx in scplxes:
        cplx_margin = 0.0
        for cluster in cplx:
            node_tmp = nodes[cluster]
            cplx_margin += avg_margin[node_tmp].detach().cpu().numpy().mean()
        cplx_margin /= len(cplx)
        scplx_margins.append((cplx, cplx_margin))
    scplx_margins.sort(key=lambda x:x[1])
    relevant_prompts = []
    print(len(scplx_margins))
    for i in scplx_margins[k:]:
        for clusters in i[0]:
            #print(nodes[cluster])
            relevant_prompts.extend(nodes[clusters])
    relevant_prompts = set(relevant_prompts)
    return np.array(imagenet_templates)[list(relevant_prompts)].tolist()

In [18]:
smart_prompt_embs = []
for c, G, margin_array in zip(imagenette_classes, graphs, avg_margins):
    new_prompts = filter_prompts(G, margin_array, c, k=5)
    smart_prompt_embs.append(get_avg_embedding_per_class(new_prompts, c))

26
['a bad photo of a {}.', 'a photo of many {}.', 'a sculpture of a {}.', 'a photo of the hard to see {}.', 'a low resolution photo of the {}.', 'a rendering of a {}.', 'graffiti of a {}.', 'a bad photo of the {}.', 'a cropped photo of the {}.', 'a tattoo of a {}.', 'a photo of a hard to see {}.', 'a bright photo of a {}.', 'a photo of a clean {}.', 'a photo of a dirty {}.', 'a dark photo of the {}.', 'a drawing of a {}.', 'a photo of my {}.', 'a photo of the cool {}.', 'a close-up photo of a {}.', 'a black and white photo of the {}.', 'a painting of the {}.', 'a painting of a {}.', 'a pixelated photo of the {}.', 'a sculpture of the {}.', 'a bright photo of the {}.', 'a cropped photo of a {}.', 'a plastic {}.', 'a jpeg corrupted photo of a {}.', 'a blurry photo of the {}.', 'a photo of the {}.', 'a good photo of the {}.', 'a rendering of the {}.', 'a photo of one {}.', 'a doodle of a {}.', 'a close-up photo of the {}.', 'a photo of a {}.', 'a sketch of a {}.', 'a doodle of the {}.', 

RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 23.65 GiB total capacity; 17.93 GiB already allocated; 8.31 MiB free; 18.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
smart_zero_wts = torch.stack(smart_prompt_embs, dim=0).squeeze(1).T

In [ ]:
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(loader):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ smart_zero_wts

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

In [ ]:
list(relevant_prompts)